In [1]:
import numpy as np

In [110]:
class Finger():
    def __init__(self, start, succ_id):
        # stores id only
        self.start = start
        self.succ_id = succ_id

In [72]:
class ChordNode:
    def __init__(self, id, m_bits, chord_circle, predecessor_id):
        self.id = id
        self.m = m_bits
        self.finger_table = []
        self.chord_circle = chord_circle
        self.chord_circle.add_node(self)
        self.predecessor_id = predecessor_id

    def successor(self):
        successor_id = self.finger_table[0].succ_id
        return self.chord_circle.get_node(successor_id)

    def successor_id(self):
        return self.finger_table[0].succ_id

    def predecessor(self):
        return self.chord_circle.get_node(self.predecessor_id)

    def find_successor(self, id):
        if self.id == id:
            return self

        node = self.find_predecessor(id)
        return node.successor()

    def find_predecessor(self, id):
        if self.id == id:
            return self.predecessor()

        node = self
        while not self.__check_interval(id, node.id, node.successor_id(), b_border=True):
            node = node.closest_preceding_finger(id)
        return node

    def closest_preceding_finger(self, id):
        for i in range(len(self.finger_table) - 1, -1, -1):
            finger_id = self.finger_table[i].succ_id
            if self.__check_interval(finger_id, self.id, id):
                node = self.chord_circle.get_node(finger_id)
                if node is None:
                    self.finger_table[i].succ_id = self.id
                    continue
                return self.chord_circle.get_node(finger_id)
        return self

    def join(self, other_node):
        print(f"Node with id = {self.id} started joining the circle.")
        self.prepare_finger_table()
        if other_node is not None:
            self.init_finger_table(other_node)
            self.update_others()
        else:
            # node is alone in circle
            self.predecessor_id = self.id
            for i in range(self.m):
                self.finger_table[i].succ_id = self.id
        print("join finished.")
                

    def prepare_finger_table(self):
        self.finger_table = []
        for i in range(self.m):
            finger = Finger(None, None)
            finger.start = (self.id + 2**i) % 2 ** self.m
            self.finger_table.append(finger)

    # fill in finger table of this node
    def init_finger_table(self, other_node):
        first_finger_start = self.finger_table[0].start
        successor = other_node.find_successor(first_finger_start)
        self.finger_table[0].succ_id = successor.id
        self.predecessor_id = successor.predecessor_id
        successor.predecessor_id = self.id

        for i in range(self.chord_circle.m_bits - 1):
            pre_finger = self.finger_table[i]
            finger = self.finger_table[i + 1]
            if self.__check_interval(finger.start, self.id, pre_finger.succ_id, a_border=True):
                finger.succ_id = pre_finger.succ_id
            else:
                finger.succ_id = other_node.find_successor(finger.start).id

    # update nodes whose fingers refers to this node
    def update_others(self, removed_id=None):
        for i in range(self.m):
            # last node whose i-th finger might be self node
            pred = (self.id - 2**i + 1) % 2**self.m
            p = self.find_predecessor(pred)
            if removed_id is not None and p.id == removed_id:
                p = self.find_predecessor(removed_id)

            p.update_finger_table(self.id, i)

    def update_finger_table(self, s, i):
        # s - new id that should be as i-th finger of self node
        if s == self.id:
            return
        finger = self.finger_table[i]
        # if node was disconnected
        not_accessible = self.chord_circle.get_node(finger.succ_id) is None
        if self.__check_interval(s, self.id, finger.succ_id, b_border=True,  a_border=True) or not_accessible:
            print(f'  At node with id={self.id} updated finger[{i}]: {finger.succ_id} -> {s}')
            finger.succ_id = s
            p_node = self.chord_circle.get_node(self.predecessor_id)
            p_node.update_finger_table(s, i)

    def disconnect(self):
        self.chord_circle.remove_node(self.id)
        # Update immediate neighbours
        self.successor().predecessor_id = self.predecessor().id
        self.predecessor().finger_table[0].succ_id = self.successor().id
        self.successor().update_others(self.id)
        print(f"Node with id = {self.id} was disconnected.")

    def __check_interval(self, x, a, b, a_border=False, b_border=False):
        if a_border and not b_border:
            if x == a:
                return True
            else:
                return self.__inbetween(x, a, b)
        elif b_border and not a_border:
            if x == b:
                return True
            else:
                return self.__inbetween(x, a, b)
        elif a_border and b_border:
            if x == b or x == a:
                return True
            else:
                return self.__inbetween(x, a, b)
        else:
            return self.__inbetween(x, a, b)

    def __inbetween(self, x, a, b):
        # borders excluded
        if a == b:
            return True
        if b < a:
            return a < x <= 2 ** self.m or 0 <= x < b
        else:
            return a < x < b

    def print_finger_table(self):
        print(f'Node id = {self.id}')
        print(f'predec = {self.predecessor_id}, succ = {self.successor().id}')
        print('start   succ.')
        for i in self.finger_table:
            print(f'{i.start}        {i.succ_id}')
        print('\n')

In [73]:
class ChordCircle:
    def __init__(self, m_bits):
        self.m_bits = m_bits
        self.chord_circle = {}

    def add_node(self, node: ChordNode):
        self.chord_circle[node.id] = node
        print(f"Node with id = {node.id} was created.")

    def get_node(self, id) -> ChordNode:
        return self.chord_circle.get(id)

    def remove_node(self, id):
        self.chord_circle.pop(id)

    def init_chord_circle(self, circle):
        for node in circle:
            self.chord_circle[node.id] = node

    def generate_id(self):
        return np.random.choice([i for i in range(0, 2**self.m_bits) 
                                 if i not in list(self.chord_circle.keys())])

    def generate_node(self):
        new_id = self.generate_id()
        new_node = ChordNode(id=new_id,
                             m_bits=self.m_bits,
                             chord_circle=self,
                            predecessor_id=None)
        return new_node


    def print_circle(self):
        for i in self.chord_circle:
            self.chord_circle[i].print_finger_table()

In [99]:
circle1 = ChordCircle(3)

In [100]:
node0 = ChordNode(
    id=0,
    m_bits=3,
    chord_circle=circle1,
    predecessor_id=None)

node0.join(None)

Node with id = 0 was created.
Node with id = 0 started joining the circle.
join finished.


In [101]:
circle1.print_circle()

Node id = 0
predec = 0, succ = 0
start   succ.
1        0
2        0
4        0




In [102]:
node1 = ChordNode(
    id=1,
    m_bits=3,
    chord_circle=circle1, 
    predecessor_id=None)

node1.join(node0)

Node with id = 1 was created.
Node with id = 1 started joining the circle.
  At node with id=0 updated finger[0]: 0 -> 1
join finished.


In [103]:
circle1.print_circle()

Node id = 0
predec = 1, succ = 1
start   succ.
1        1
2        0
4        0


Node id = 1
predec = 0, succ = 0
start   succ.
2        0
3        0
5        0




In [104]:
node2 = ChordNode(
    id=2,
    m_bits=3,
    chord_circle=circle1, 
    predecessor_id=None)

node2.join(node1)

Node with id = 2 was created.
Node with id = 2 started joining the circle.
  At node with id=1 updated finger[0]: 0 -> 2
  At node with id=0 updated finger[1]: 0 -> 2
join finished.


In [105]:
circle1.print_circle()

Node id = 0
predec = 2, succ = 1
start   succ.
1        1
2        2
4        0


Node id = 1
predec = 0, succ = 2
start   succ.
2        2
3        0
5        0


Node id = 2
predec = 1, succ = 0
start   succ.
3        0
4        0
6        0




In [106]:
node3 = circle1.generate_node()
node3.join(node0)

Node with id = 4 was created.
Node with id = 4 started joining the circle.
  At node with id=2 updated finger[0]: 0 -> 4
  At node with id=2 updated finger[1]: 0 -> 4
  At node with id=1 updated finger[1]: 0 -> 4
  At node with id=0 updated finger[2]: 0 -> 4
join finished.


In [107]:
circle1.print_circle()

Node id = 0
predec = 4, succ = 1
start   succ.
1        1
2        2
4        4


Node id = 1
predec = 0, succ = 2
start   succ.
2        2
3        4
5        0


Node id = 2
predec = 1, succ = 4
start   succ.
3        4
4        4
6        0


Node id = 4
predec = 2, succ = 0
start   succ.
5        0
6        0
0        0




In [108]:
node2.disconnect()

  At node with id=1 updated finger[0]: 4 -> 4
  At node with id=1 updated finger[1]: 4 -> 4
  At node with id=0 updated finger[1]: 0 -> 4
  At node with id=0 updated finger[2]: 4 -> 4
Node with id = 2 was disconnected.


In [109]:
circle1.print_circle()

Node id = 0
predec = 4, succ = 1
start   succ.
1        1
2        4
4        4


Node id = 1
predec = 0, succ = 4
start   succ.
2        4
3        4
5        0


Node id = 4
predec = 1, succ = 0
start   succ.
5        0
6        0
0        0


